# COMP34812 Natural Language Understanding Courseworklow key lemming an stemming


## Install required packages

In [ ]:
!pip install pandas nltk numpy matplotlib scikit-learn sentencepiece


[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import regex as re
import numpy as np
import nltk
import os
nltk.download('punkt')
nltk.download('stopwords')

from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\zaccu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\zaccu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
if not os.path.exists('glove_embeddings'):
  !wget https://nlp.stanford.edu/data/glove.6B.zip
  !unzip glove.6B.zip -d glove_embeddings

## Load dataset

In [4]:
dev_set = pd.read_csv('dev.csv')
dev_set.head()

,premise,hypothesis,label
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0
4,I know that many of you are interested in addr...,The problems must be addressed,1


In [5]:
train_set = pd.read_csv('train.csv')
train_set.head()

,premise,hypothesis,label
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1


In [6]:
stop_words = nltk.corpus.stopwords.words('english')
lemmatizer = nltk.stem.WordNetLemmatizer()

def clean_text(text):
    text = str(text)

    text = text.lower()

    text = re.sub(r'[^\w\s]', ' ', text)

    text = nltk.word_tokenize(text)

    processed = []
    for word in text:
        if word in stop_words:
            continue

        word = lemmatizer.lemmatize(word)

        word = word.strip()

        if len(word) < 2:
            continue

        processed.append(word)

    return processed

In [7]:
dev_set['premise_tokens'] = dev_set['premise'].apply(clean_text)
dev_set['hypothesis_tokens'] = dev_set['hypothesis'].apply(clean_text)

train_set['premise_tokens'] = train_set['premise'].apply(clean_text)
train_set['hypothesis_tokens'] = train_set['hypothesis'].apply(clean_text)

In [8]:
dev_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,"By starting at the soft underbelly, the 16,000...","General Nelson A. Miles had 30,000 troops in h...",0,"[starting, soft, underbelly, 16, 000, troop, g...","[general, nelson, mile, 30, 000, troop, attack]"
1,"The class had broken into a light sweat, but w...",The class grew more tense as time went on.,1,"[class, broken, light, sweat, gasping, air]","[class, grew, tense, time, went]"
2,"Samson had his famous haircut here, but he wou...",It was unknown where exactly within the town S...,1,"[samson, famous, haircut, would, find, hard, r...","[unknown, exactly, within, town, samson, recei..."
3,A man with a black shirt holds a baby while a ...,A darkly dressed man passes a crying baby to a...,0,"[man, black, shirt, hold, baby, blue, shirted,...","[darkly, dressed, man, pass, cry, baby, man, l..."
4,I know that many of you are interested in addr...,The problems must be addressed,1,"[know, many, interested, addressing, issue, le...","[problem, must, addressed]"


In [9]:
train_set.head()

,premise,hypothesis,label,premise_tokens,hypothesis_tokens
0,yeah i don't know cut California in half or so...,Yeah. I'm not sure how to make that fit. Maybe...,1,"[yeah, know, cut, california, half, something]","[yeah, sure, make, fit, maybe, could, cut, cal..."
1,actual names will not be used,"For the sake of privacy, actual names are not ...",1,"[actual, name, used]","[sake, privacy, actual, name, used]"
2,The film was directed by Randall Wallace.,The film was directed by Randall Wallace and s...,1,"[film, directed, randall, wallace]","[film, directed, randall, wallace, star, mel, ..."
3,"""How d'you know he'll sign me on?""Anse studie...",Anse looked at himself in a cracked mirror.,1,"[know, sign, anse, studied, unkempt, clean, re...","[anse, looked, cracked, mirror]"
4,In the light of the candles his cheeks looked ...,Drew regarded his best friend and noted that i...,1,"[light, candle, cheek, looked, even, hollow, t...","[drew, regarded, best, friend, noted, light, l..."


Dataset analysis

In [10]:
# Labels = dev_set['label'].unique()
# Labels

# def get_word_frequency(data):
#     word_freq = {}
#     for row in data:
#         for word in row:
#             if word in word_freq:
#                 word_freq[word] += 1
#             else:
#                 word_freq[word] = 1
#     return word_freq

# word_freq = get_word_frequency(train_set['premise_tokens'] + train_set['hypothesis'])

# # nltk FreqDist
# from nltk import FreqDist

# fdist = FreqDist(word_freq)
# fdist

# BPE tokinizer

In [ ]:
import sentencepiece as spm

# Train BPE on training dataset (premises + hypotheses combined)
spm.SentencePieceTrainer.train(
    input= concat(train_set['premise'],train_set['hypothosis']) ,  # A file where each line is a sentence from the dataset
    model_prefix="bpe", 
    vocab_size=10000,  # Adjust as needed
    model_type="bpe"
)

# Load trained BPE tokenizer
#bpe = spm.SentencePieceProcessor(model_file="bpe.model")


# Glove embeddings and tfif wieghted word emeddings plus named eneitiy 

In [11]:
glove = "./glove_embeddings/glove.6B.200d.txt"
def load_glove(glove_file):
    embeddings_dict = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype=np.float32)  # <-- Convert to float32
            embeddings_dict[word] = vector
    return embeddings_dict

embedding_dim = 200
loaded_glove = load_glove(glove)



In [14]:
def sentence_embedding(tokens, embeddings_dict, embedding_dim):
    valid_embeddings = [embeddings_dict[token] for token in tokens if token in embeddings_dict]
    if not valid_embeddings:
        # Return zero-vector if no embeddings found
        return np.zeros(embedding_dim)
    sentence_emb = np.mean(valid_embeddings, axis=0)
    return sentence_emb

def pairwise_embedding(premise_tokens, hypothesis_tokens, embeddings_dict,embedding_dim):
    premise_emb = sentence_embedding(premise_tokens, embeddings_dict,embedding_dim)
    hypothesis_emb = sentence_embedding(hypothesis_tokens, embeddings_dict,embedding_dim)
    # Concatenate multiple useful features
    combined_emb = np.concatenate([
        premise_emb,
        hypothesis_emb,
        np.abs(premise_emb - hypothesis_emb), # capture difference
        premise_emb * hypothesis_emb           # capture interactions
    ]).astype(np.float32)

    return combined_emb



In [20]:
import numpy as np
from tqdm import tqdm
tqdm.pandas()

# Convert list of numpy arrays into a single 2D numpy array
train_embeddings = np.stack(train_set.progress_apply(
    lambda x: pairwise_embedding(x['premise_tokens'], x['hypothesis_tokens'], loaded_glove, embedding_dim=200), axis=1))

dev_embeddings = np.stack(dev_set.progress_apply(
    lambda x: pairwise_embedding(x['premise_tokens'], x['hypothesis_tokens'], loaded_glove, embedding_dim=200), axis=1))


100%|██████████| 6736/6736 [00:00<00:00, 15633.66it/s]


# Traditional Approach

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],  # Regularization strength
    'solver': ['lbfgs', 'liblinear'],  # Different solvers for logistic regression
    'max_iter': [500, 1000, 2000]  # More iterations for convergence
}

clf = GridSearchCV(LogisticRegression(), param_grid, cv=3, scoring='f1_macro', verbose=1, n_jobs=-1)
clf.fit(train_embeddings, train_set["label"].values)  # Train on enhanced embeddings

# Print Best Parameters
print("Best Parameters:", clf.best_params_)

# Evaluate on validation set
preds = clf.best_estimator_.predict(dev_embeddings)
print(classification_report(dev_set['label'].values, preds, target_names=['entailment', 'contradiction']))


Fitting 2 folds for each of 30 candidates, totalling 60 fits
Best Parameters: {'C': 0.1, 'max_iter': 500, 'solver': 'lbfgs'}
               precision    recall  f1-score   support

   entailment       0.59      0.51      0.55      3258
contradiction       0.59      0.66      0.62      3478

     accuracy                           0.59      6736
    macro avg       0.59      0.59      0.58      6736
 weighted avg       0.59      0.59      0.59      6736

